In [ ]:
def VegaLite(spec):
    bundle = {}
    bundle['application/vnd.vegalite.v4+json'] = spec
    display(bundle, raw=True)
from csv import DictReader

pointcalls_observations_Marseille = []
with open('../../data/navigo_all_pointcalls.csv', newline='') as csvfile:
    reader = DictReader(csvfile)
    for row in reader:
        if row['date_fixed'].split('-')[0] == '1789' \
        and row['pointcall_function'] == "O" \
        and (row["source_suite"] == "Registre du petit cabotage (1786-1787)" \
             or row["source_suite"] == "la Santé registre de patentes de Marseille") \
        and row['net_route_marker'] != "Q":
           # and row['toponyme_fr'] == 'Marseille' \
            pointcalls_observations_Marseille.append(row)

In [ ]:
import pandas as pd

In [ ]:
pointcalls_before_Marseille = []
with open('../../data/navigo_all_pointcalls.csv', newline='') as csvfile:
    reader = DictReader(csvfile)
    for row in reader:
        if row['date_fixed'].split('-')[0] == '1789' \
        and (row["source_suite"] == "Registre du petit cabotage (1786-1787)" \
             or row["source_suite"] == "la Santé registre de patentes de Marseille") \
        and row['net_route_marker'] != "Q":
           # and row['toponyme_fr'] == 'Marseille' \
            pointcalls_before_Marseille.append(row)

In [ ]:
from collections import Counter
from pprint import pprint
toponyme_counter_corse = Counter([point["toponyme_fr"] for point in pointcalls_before_Marseille if point["pointcall_province"] == "Corse"])
toponyme_counter_roussillon = Counter([point["toponyme_fr"] for point in pointcalls_before_Marseille if point["pointcall_province"] == "Roussillon"])


In [ ]:
rank_Marseille = {}
for row in pointcalls_observations_Marseille:
    rank_Marseille[row["source_doc_id"]] = row["pointcall_rank_dedieu"]

In [ ]:
ranks_smaller_than_Marseille = []
for row in pointcalls_before_Marseille:
    if row["source_doc_id"] in rank_Marseille:
        max_rank = rank_Marseille[row["source_doc_id"]]
        if row["pointcall_rank_dedieu"] < max_rank:
            ranks_smaller_than_Marseille.append(row)

In [ ]:
for row in ranks_smaller_than_Marseille:
    if row["toponyme_fr"] in toponyme_counter_corse:
        row["toponyme_fr"] = "Corse"
    elif row["toponyme_fr"] in toponyme_counter_roussillon:
        row["toponyme_fr"] = "Roussillon"

In [ ]:
import requests
import csv
from pprint import pprint

def get_online_csv(url):
  results = []
  with requests.Session() as s:
      download = s.get(url)
      decoded_content = download.content.decode('utf-8')
      reader = csv.DictReader(decoded_content.splitlines(), delimiter=',')
      for row in reader:
        results.append(dict(row))
  return results

sorties_g5 = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRNAeIEFhB_RTm2xBgeuXl5oMtNIrGhWT6uCB2S9wEUblwDidRBwv9dp8D0S-YIPUyoASaG2p-NgfWD/pub?output=csv'
sorties_g5 = get_online_csv(sorties_g5)


In [ ]:
from csv import DictReader
import pandas as pd

with open('../../data/navigo_all_pointcalls.csv', newline='', encoding='utf8') as csvfile:
    pointcalls = pd.read_csv(csvfile, sep=',')

In [ ]:
data = pointcalls[['toponyme_fr', 'pointcall_province']]
arriere_pays = ["Corse", "Provence", "Languedoc", "Roussillon"]
subset_data = data[data['pointcall_province'].isin(arriere_pays)]
subset_data.drop_duplicates(keep = 'first', inplace=True)
subset_data

In [ ]:
df_sorties_g5 = pd.DataFrame(sorties_g5, columns =['annee', 
                                                  'nb_conges_cr',
                                                  'nb_conges_inputdone',
                                                  'nb_conges_sante',
                                                  'nb_longcours_marseille',
                                                  'nb_petitcabotage',
                                                  'toponyme_standard_fr',
                                                  'uhgs_id'
                                                  ])

sortie_arriere_pays = pd.merge(df_sorties_g5, subset_data, left_on='toponyme_standard_fr', right_on='toponyme_fr')

In [ ]:
sortie_arriere_pays_1789 = sortie_arriere_pays[sortie_arriere_pays['annee'] == '1789']
sortie_arriere_pays_1789 = sortie_arriere_pays_1789[['nb_conges_cr', 'toponyme_fr', 'pointcall_province']]
sortie_arriere_pays_1789 

In [ ]:
from IPython.display import display

def VegaLite(spec):
    bundle = {}
    bundle['application/vnd.vegalite.v4+json'] = spec
    display(bundle, raw=True)

In [ ]:
dict_sortie_arriere_pays_1789 = sortie_arriere_pays_1789.to_dict('records')

In [ ]:
departures = {"Corse" : 0, "Roussillon" : 0}

for i, row in sortie_arriere_pays_1789.iterrows():
    if row["pointcall_province"] == "Corse":
        departures["Corse"] += int(row["nb_conges_cr"])
    elif row["pointcall_province"] == "Roussillon":
        departures["Roussillon"] += int(row["nb_conges_cr"])
    else:
        departures[row["toponyme_fr"]] = int(row["nb_conges_cr"])

In [ ]:
sortie_arriere_pays_1789

In [ ]:
polarization_ratios = defaultdict(dict)
for departure, value in departures.items():
    polarization_ratios[departure]["departure_count"] = value
    polarization_ratios[departure]["departure_port"] = departure
    polarization_ratios[departure]["arrival_count"] = 0

for row in ranks_smaller_than_Marseille:
    if row["toponyme_fr"] in departures:
        polarization_ratios[row["toponyme_fr"]]["arrival_count"] += 1

In [ ]:
list_polarisation_ratios = []
for k, v in polarization_ratios.items():
    list_polarisation_ratios.append({"port": k, "value": v["departure_count"], "type": "departure"})
    list_polarisation_ratios.append({"port": k, "value": v["arrival_count"], "type": "arrival"})

In [ ]:
viz_df = pd.DataFrame(list_polarisation_ratios)
viz_df = viz_df.sort_values(["type", "value"])

In [ ]:
viz_df = viz_df[viz_df.port != "Marseille"]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="whitegrid")

# Draw a nested barplot by species and sex
g = sns.catplot(
    data=viz_df, kind="bar",
    x="value", y="port", hue="type", orient="h",
    height=6
)
g.set_axis_labels("nb de bateaux", "Ports")

# plt.savefig("../../data/navigo_all_pointcalls.csv")
plt.show()

In [ ]:
viz_df[viz_df.port == "Bormes"]

In [ ]:
viz_df_arrival = viz_df[viz_df.type == "arrival"]
viz_df_departure = viz_df[viz_df.type == "departure"]
viz_df_ratios = viz_df_departure.merge(viz_df_arrival, on="port")
viz_df_ratios["ratio"] = ((viz_df_ratios.value_y.astype(int) / viz_df_ratios.value_x.astype(int)) * 100).round().astype(int)
viz_df_ratios.sort_values(by=['ratio'], inplace=True)

In [ ]:
# Suppression des villes avec des ratios absurdes 
# 10 bateaux partent et 100 bateaux arrivent à Marseille

viz_df_ratios = viz_df_ratios[viz_df_ratios.ratio <= 101]
viz_df_ratios

In [ ]:
# Draw a nested barplot by species and sex
g = sns.catplot(
    data=viz_df_ratios,
    x="ratio", y="port", orient="h", color="grey",
    kind="bar",
    height=8.27, aspect=11.7/8.27,
    
)

ax = g.facet_axis(0, 0)

# iterate through the axes containers
for c in ax.containers:
    ax.bar_label(c, fmt='%.0f%%', padding=4)

g.set_axis_labels("Ratio de bateaux arrivant à Marseille sur le nombre de bateaux au départ", "Ports")

# plt.savefig("ratios.png")
plt.show()